## 1 - Importing related libraries and functions
In this section we import necessary libraries and functions:


In [16]:
from delaunay import extract_features
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler

## 2 - Load and preprocess the dataset
In this section we first import our designated dataset which is `CK+` dataset, read ,and load it in an array


In [2]:
dataset_dir = "dataset"

label_map = {
    "anger": 0,
    "contempt": 1,
    "disgust": 2,
    "fear": 3,
    "happy": 4,
    "sadness": 5,
    "surprise": 6
}
# Load and preprocess the dataset
def load_ckplus_dataset(dataset_dir):
    images = []
    labels = []
    for subdir in os.listdir(dataset_dir):
        if subdir in label_map:  # Check if directory name is a valid label
            label = label_map[subdir]  # Get integer label from the mapping
            for filename in os.listdir(os.path.join(dataset_dir, subdir)):
                if filename.endswith(".png"):  # Load only image files (assuming images are in PNG format)
                    img_path = os.path.join(dataset_dir, subdir, filename)
                    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Convert to grayscale
                    img = cv2.resize(img, (100, 100))  # Resize image
                    images.append(img)
                    labels.append(label)

    return images, labels

images, labels = load_ckplus_dataset(dataset_dir)


## 3 - Compare the results from raw image classification and delaunay triangulation method
### 3.1 - Train the raw dataset
As we know the dataset is in grayscale, we need to get the brightness level of each pixel and train our models on each image   
then we will report the results:
> Note: We have used standard scalar normalization that leads to better results


#### Pre-Process data:

In [3]:
# Initialize the scalar
scalar = StandardScaler()

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

X_train = np.array([img.flatten() for img in X_train])
X_test = np.array([img.flatten() for img in X_test])

X_norm = scalar.fit_transform(X_train)
X_norm_test = scalar.fit_transform(X_test)


#### Training phase: 
1. SVM Model:

In [4]:
# Train the SVM classifier
svm_classifier = SVC(kernel='rbf', C=1.0, gamma='scale')
svm_classifier.fit(X_norm, y_train)

# Evaluate the classifier
y_pred = svm_classifier.predict(X_norm_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

Accuracy: 0.8934010152284264
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.87      0.85        23
           1       0.64      0.78      0.70         9
           2       0.86      0.88      0.87        43
           3       0.82      0.93      0.88        15
           4       1.00      0.95      0.98        43
           5       1.00      0.68      0.81        19
           6       0.91      0.96      0.93        45

    accuracy                           0.89       197
   macro avg       0.87      0.87      0.86       197
weighted avg       0.90      0.89      0.89       197


2. K-Nearest-Neighbors model:

In [5]:
# Train the KNN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=5)
knn_classifier.fit(X_norm, y_train)

# Evaluate the classifier
y_pred = knn_classifier.predict(X_norm_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

Accuracy: 0.3604060913705584
Classification Report:
               precision    recall  f1-score   support

           0       0.21      0.43      0.28        23
           1       0.50      0.44      0.47         9
           2       0.37      0.40      0.38        43
           3       0.29      0.13      0.18        15
           4       0.34      0.33      0.33        43
           5       0.24      0.21      0.22        19
           6       0.67      0.44      0.53        45

    accuracy                           0.36       197
   macro avg       0.37      0.34      0.34       197
weighted avg       0.40      0.36      0.37       197


3. Naive Bayesian Classification:

In [6]:
# Train the GNB classifier
gnb_classifier = GaussianNB()
gnb_classifier.fit(X_norm, y_train)

# Evaluate the classifier
y_pred = gnb_classifier.predict(X_norm_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

Accuracy: 0.467005076142132
Classification Report:
               precision    recall  f1-score   support

           0       0.10      0.17      0.13        23
           1       0.21      0.56      0.30         9
           2       0.43      0.47      0.45        43
           3       0.00      0.00      0.00        15
           4       0.66      0.63      0.64        43
           5       0.40      0.11      0.17        19
           6       0.89      0.76      0.82        45

    accuracy                           0.47       197
   macro avg       0.39      0.38      0.36       197
weighted avg       0.50      0.47      0.47       197


4. Decision Tree:

In [7]:
# Train the Tree classifier
tree_classifier = DecisionTreeClassifier() 
tree_classifier.fit(X_norm, y_train)

# Evaluate the classifier
y_pred = tree_classifier.predict(X_norm_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

Accuracy: 0.817258883248731
Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.78      0.78        23
           1       0.50      0.44      0.47         9
           2       0.94      0.70      0.80        43
           3       0.57      0.87      0.68        15
           4       0.93      0.88      0.90        43
           5       0.76      0.84      0.80        19
           6       0.86      0.93      0.89        45

    accuracy                           0.82       197
   macro avg       0.76      0.78      0.76       197
weighted avg       0.83      0.82      0.82       197


5. Quadratic Discriminant Analysis:

In [8]:
# Train the QDA classifier
qda_classifier = QuadraticDiscriminantAnalysis()
qda_classifier.fit(X_norm, y_train)

# Evaluate the classifier
y_pred = qda_classifier.predict(X_norm_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Accuracy: 0.48223350253807107
Classification Report:
               precision    recall  f1-score   support

           0       0.43      0.43      0.43        23
           1       0.30      0.67      0.41         9
           2       0.51      0.51      0.51        43
           3       0.75      0.20      0.32        15
           4       0.53      0.49      0.51        43
           5       0.35      0.37      0.36        19
           6       0.55      0.58      0.57        45

    accuracy                           0.48       197
   macro avg       0.49      0.46      0.44       197
weighted avg       0.51      0.48      0.48       197


6. Random Forest:

In [9]:
# Train the rf classifier
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_norm, y_train)

# Evaluate the classifier
y_pred = rf_classifier.predict(X_norm_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

Accuracy: 0.9949238578680203
Classification Report:
               precision    recall  f1-score   support

           0       0.96      1.00      0.98        23
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        43
           3       1.00      1.00      1.00        15
           4       1.00      1.00      1.00        43
           5       1.00      0.95      0.97        19
           6       1.00      1.00      1.00        45

    accuracy                           0.99       197
   macro avg       0.99      0.99      0.99       197
weighted avg       1.00      0.99      0.99       197


### 3.2 - Train the on dataset with triangulation
As implemented in the corresponding class, we need to detect landmarks of each image and use the detected points to train our model and give the results: 
> Note: We have used standard scalar normalization that leads to better results


We first build a function to process landmarks for rach image and save it to save time:

In [17]:
def extract_landmark_features(images):
    landmarks_list = []
    feature_lengths = []
    landmarks_vector = []
    
    for image in images:
        features, lands  = extract_features(image)
        landmarks_list.append(features.tolist())
        landmarks_vector.append(np.array(lands).flatten())
        feature_lengths.append(len(features))

    max_features = max(feature_lengths)

    for i in range(len(images)):
        current_length = len(landmarks_list[i])
        if current_length < max_features:
            padding = [0] * (max_features - current_length)
            landmarks_list[i].extend(padding)

    head = np.arange(max_features)
    data_combined = pd.DataFrame(landmarks_list, columns=head)
    data_combined.to_csv('features.csv', index=False)
    data_landmarks = pd.DataFrame(landmarks_vector)
    data_landmarks.to_csv('landmarks.csv',index=False)
    
    return np.array(landmarks_list)

landmark_features = extract_landmark_features(images)

68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
6

### Pre-Process data:
we use previously stored data for training:

In [29]:
df = pd.read_csv('landmarks.csv')
landmark_features = df.values

# Initialize the scalar
scalar = StandardScaler()

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(landmark_features, labels, test_size=0.1, random_state=42)

X_train = np.array([img.flatten() for img in X_train])
X_test = np.array([img.flatten() for img in X_test])

X_norm = scalar.fit_transform(X_train)
X_norm_test = scalar.transform(X_test)

landmark_features

array([[10, 45, 11, ..., 79, 46, 79],
       [10, 45, 10, ..., 78, 46, 78],
       [11, 44, 11, ..., 79, 47, 79],
       ...,
       [ 7, 51,  8, ..., 89, 46, 89],
       [ 6, 51,  7, ..., 90, 47, 90],
       [ 6, 53,  7, ..., 93, 46, 92]], dtype=int64)

#### Training phase: 
1. SVM Model:

In [30]:
# Train the SVM classifier
svm_classifier_tri = SVC(kernel='rbf', C=1.0, gamma='scale')
svm_classifier_tri.fit(X_norm, y_train)

# Evaluate the classifier
y_pred = svm_classifier_tri.predict(X_norm_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

Accuracy: 0.9292929292929293
Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.92      0.89        13
           1       0.50      1.00      0.67         4
           2       0.96      1.00      0.98        23
           3       1.00      1.00      1.00         3
           4       1.00      0.95      0.97        20
           5       1.00      0.89      0.94         9
           6       1.00      0.85      0.92        27

    accuracy                           0.93        99
   macro avg       0.90      0.94      0.91        99
weighted avg       0.95      0.93      0.93        99


2. K-Nearest-Neighbors model:

In [31]:
# Train the KNN classifier
knn_classifier_tri = KNeighborsClassifier(n_neighbors=5)
knn_classifier_tri.fit(X_norm, y_train)

# Evaluate the classifier
y_pred = knn_classifier_tri.predict(X_norm_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

Accuracy: 0.7676767676767676
Classification Report:
               precision    recall  f1-score   support

           0       0.61      0.85      0.71        13
           1       0.75      0.75      0.75         4
           2       0.75      0.78      0.77        23
           3       0.50      0.67      0.57         3
           4       0.77      0.85      0.81        20
           5       0.67      0.44      0.53         9
           6       1.00      0.78      0.88        27

    accuracy                           0.77        99
   macro avg       0.72      0.73      0.72        99
weighted avg       0.79      0.77      0.77        99


3. Naive Bayesian Classification:

In [32]:
# Train the GNB classifier
gnb_classifier_tri = GaussianNB()
gnb_classifier_tri.fit(X_norm, y_train)

# Evaluate the classifier
y_pred = gnb_classifier_tri.predict(X_norm_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

Accuracy: 0.696969696969697
Classification Report:
               precision    recall  f1-score   support

           0       0.47      0.54      0.50        13
           1       0.30      0.75      0.43         4
           2       0.71      0.74      0.72        23
           3       0.40      0.67      0.50         3
           4       0.88      0.75      0.81        20
           5       0.67      0.44      0.53         9
           6       0.95      0.78      0.86        27

    accuracy                           0.70        99
   macro avg       0.63      0.67      0.62        99
weighted avg       0.75      0.70      0.71        99


4. Decision Tree:

In [33]:
# Train the Tree classifier
tree_classifier_tri = DecisionTreeClassifier() 
tree_classifier_tri.fit(X_norm, y_train)

# Evaluate the classifier
y_pred = tree_classifier_tri.predict(X_norm_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

Accuracy: 0.8686868686868687
Classification Report:
               precision    recall  f1-score   support

           0       0.59      0.77      0.67        13
           1       1.00      0.75      0.86         4
           2       0.83      0.87      0.85        23
           3       0.75      1.00      0.86         3
           4       1.00      0.95      0.97        20
           5       0.89      0.89      0.89         9
           6       1.00      0.85      0.92        27

    accuracy                           0.87        99
   macro avg       0.87      0.87      0.86        99
weighted avg       0.89      0.87      0.87        99


5. Quadratic Discriminant Analysis:

In [34]:
# Train the QDA classifier
qda_classifier_tri = QuadraticDiscriminantAnalysis()
qda_classifier_tri.fit(X_norm, y_train)

y_pred = qda_classifier_tri.predict(X_norm)
accuracy = accuracy_score(y_train, y_pred)
print("train Accuracy:", accuracy)

# Evaluate the classifier
y_pred = qda_classifier_tri.predict(X_norm_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

train Accuracy: 0.9977324263038548
Accuracy: 0.47474747474747475
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        13
           1       0.00      0.00      0.00         4
           2       0.75      0.13      0.22        23
           3       0.00      0.00      0.00         3
           4       0.74      0.85      0.79        20
           5       0.00      0.00      0.00         9
           6       0.38      1.00      0.55        27

    accuracy                           0.47        99
   macro avg       0.27      0.28      0.22        99
weighted avg       0.43      0.47      0.36        99


C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509:

6. Random Forest:

In [35]:
# Train the rf classifier
rf_classifier_tri = RandomForestClassifier()
rf_classifier_tri.fit(X_norm, y_train)

y_pred = rf_classifier_tri.predict(X_norm)
accuracy = accuracy_score(y_train, y_pred)
print("train Accuracy:", accuracy)

# Evaluate the classifier
y_pred = rf_classifier_tri.predict(X_norm_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("test Accuracy:", accuracy)
print("Classification Report:\n", report)

train Accuracy: 1.0
test Accuracy: 0.9696969696969697
Classification Report:
               precision    recall  f1-score   support

           0       0.87      1.00      0.93        13
           1       0.80      1.00      0.89         4
           2       1.00      1.00      1.00        23
           3       1.00      1.00      1.00         3
           4       1.00      1.00      1.00        20
           5       1.00      1.00      1.00         9
           6       1.00      0.89      0.94        27

    accuracy                           0.97        99
   macro avg       0.95      0.98      0.97        99
weighted avg       0.97      0.97      0.97        99


So based on the results:
- some models
- some other


Now we introduce our own Neural Network model to outperform others:

### Neural Network:

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Flatten
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping


X_train, X_test, y_train, y_test = train_test_split(landmark_features, labels, test_size=0.1, random_state=42)


X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.1), input_shape=(982, 136, 1)),
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.1)),
    Flatten(),
    Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.4),
    Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
    Dense(7, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Early stopping monitor
early_stopping_monitor = EarlyStopping(
    monitor='val_accuracy',
    patience=15,
    restore_best_weights=True
)

# Train the model with early stopping
model.fit(X_train, y_train, epochs=50, validation_split=0.1, callbacks=[early_stopping_monitor])

# Evaluate on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
